# 2018/09/20 문제 인식 및 실험 개시

## Library 호출

In [27]:
import numpy as np #수치계산 라이브러리
import pandas as pd #구조데이터 처리 라이브러리
import matplotlib.pyplot as plt #그래프 라이브러리
%matplotlib inline

## 파일 호출

In [28]:
import pickle
#피클 파일 호출
with open('ten_raw_data.pickle','rb') as handle:
    raw_data = pickle.load(handle)
#삼성전자, SK하이닉스, 현대차, PSOCO, LG화학, NAVER, 삼성물산, SK텔레콤, 현대모비스, KB금융
#"005930","000660","005380","047050","051910","035420","028260","017670","012330","105560"

In [29]:
stock_name=['삼성전자', 'SK하이닉스', '현대차', 'PSOCO', 'LG화학', 'NAVER','삼성물산','SK텔레콤', '현대모비스', 'KB금융']
stock_num=["005930","000660","005380","047050","051910","035420","028260","017670","012330","105560"]
#dictionary key를 code에서 주식 이름으로 교체
for i in range(10):
    raw_data[stock_name[i]] = raw_data.pop(stock_num[i])

In [30]:
del raw_data['삼성물산']
del raw_data['SK텔레콤']
del raw_data['SK하이닉스']
del raw_data['현대모비스']
del raw_data['KB금융']

In [31]:
raw_data['삼성전자'].loc['2012-01-01':'2018-07-31'].head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-01-02,21400.0,21600.0,21300.0,21600.0,21.233131,13151350
2012-01-03,21860.0,22100.0,21840.0,22100.0,21.724644,16927750
2012-01-04,22100.0,22200.0,21500.0,21600.0,21.233131,17103700
2012-01-05,21460.0,21580.0,21100.0,21100.0,20.741627,17298400
2012-01-06,21120.0,21320.0,20600.0,20800.0,20.446722,18816250


In [32]:
raw_data['삼성전자'].loc['2012-01-01':'2018-07-31'].tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-07-25,46250.0,46550.0,45900.0,46150.0,46150.0,7222471
2018-07-26,46100.0,47000.0,46000.0,46900.0,46900.0,7374946
2018-07-27,46450.0,47000.0,46450.0,46900.0,46900.0,4762460
2018-07-30,46550.0,46800.0,46350.0,46500.0,46500.0,5723035
2018-07-31,46200.0,46450.0,46000.0,46250.0,46250.0,8033607


## Feature 추가

Shift 조정 코드 간략 설명

In [33]:
#수익률 계산
raw_data['삼성전자'].Close.pct_change().head(9)

Date
2010-01-04         NaN
2010-01-05    0.016069
2010-01-06    0.023114
2010-01-07   -0.033294
2010-01-08    0.009840
2010-01-11   -0.029233
2010-01-12    0.015056
2010-01-13   -0.014833
2010-01-14    0.037641
Name: Close, dtype: float64

In [34]:
#Shift조정
#5일 후의 데이터를 예측하기 위해 5일 전의 데이터를 저장
raw_data['삼성전자'].Close.pct_change().shift(5).head(9)

#2010년 1월 12일의 0.016069는 1월 5일의 수익률이다.

Date
2010-01-04         NaN
2010-01-05         NaN
2010-01-06         NaN
2010-01-07         NaN
2010-01-08         NaN
2010-01-11         NaN
2010-01-12    0.016069
2010-01-13    0.023114
2010-01-14   -0.033294
Name: Close, dtype: float64

Feature 추가

In [35]:
for i in raw_data.keys():
    """
    X
    """
    # 1일, 5일을 shift 조정한 수익률 계산
    raw_data[i] = raw_data[i].assign(sh1_pct_change = raw_data[i].Close.pct_change().shift(1))
    raw_data[i] = raw_data[i].assign(sh5_pct_change = raw_data[i].Close.pct_change().shift(5))
    # 실제 당일의 수익률 계산
    raw_data[i] = raw_data[i].assign(pct_change = raw_data[i].Close.pct_change().fillna(0))
    
    # 1일, 5일을 shift 조정한 3일 주가 이동평균 계산
    raw_data[i] = raw_data[i].assign(sh1_close_ma_3 = raw_data[i].Close.rolling(window =3).mean().shift(1))
    raw_data[i] = raw_data[i].assign(sh5_close_ma_3 = raw_data[i].Close.rolling(window =3).mean().shift(5))
    
    # 1일, 5일을 shift 조정한 7일, 21일 역사적 변동성 계산
    raw_data[i] = raw_data[i].assign(sh1_his_vol_7 = raw_data[i]["pct_change"].rolling(window = 7).std().shift(1)*(252**0.5))
    raw_data[i] = raw_data[i].assign(sh5_his_vol_7 = raw_data[i]["pct_change"].rolling(window = 7).std().shift(5)*(252**0.5))
    
    raw_data[i] = raw_data[i].assign(sh1_his_vol_21 = raw_data[i]["pct_change"].rolling(window = 21).std().shift(1)*(252**0.5))
    raw_data[i] = raw_data[i].assign(sh5_his_vol_21 = raw_data[i]["pct_change"].rolling(window = 21).std().shift(5)*(252**0.5))
   
    # 1일, 5일을 shift 조정한 거래량 계싼
    raw_data[i] = raw_data[i].assign(sh1_volume = raw_data[i].Volume.shift(1))
    raw_data[i] = raw_data[i].assign(sh5_volume = raw_data[i].Volume.shift(5))
    
    """
    Y
    """
    # 주가를 활용한 값vs평균 Y값 계산
    raw_data[i] = raw_data[i].assign(Y1 = raw_data[i].Close - raw_data[i].Close.rolling(window =7).mean().shift(1))
    raw_data[i] = raw_data[i].assign(Y1 = np.where(raw_data[i].Y1>=0, 1, 0))
    # 주가를 활용한 평균vs평균 Y값 계산
    raw_data[i] = raw_data[i].assign(Y3 = raw_data[i].Close.rolling(window=7).mean() - raw_data[i].Close.rolling(window =7).mean().shift(1))
    raw_data[i] = raw_data[i].assign(Y3 = np.where(raw_data[i].Y3>=0, 1, 0))
    
    """
    Drop
    """
    # 하나라도 NaN Value가 있는 행or열 제거
    raw_data[i] = raw_data[i].dropna(axis=0)

In [82]:
# 정확히 하루씩 밀린 모습
raw_data['삼성전자'][['Volume', 'sh1_volume']].head()

,Volume,sh1_volume
Date,,
2010-02-08,21891500,25705750.0
2010-02-09,16834200,21891500.0
2010-02-10,13174200,16834200.0
2010-02-11,13208300,13174200.0
2010-02-12,22879300,13208300.0


In [84]:
# 정확히 5일씩 밀린 모습
raw_data['삼성전자'][['Volume', 'sh5_volume']].head(8)

,Volume,sh5_volume
Date,,
2010-02-08,21891500,25037200.0
2010-02-09,16834200,19677150.0
2010-02-10,13174200,16467700.0
2010-02-11,13208300,11971700.0
2010-02-12,22879300,25705750.0
2010-02-16,7723550,21891500.0
2010-02-17,13997150,16834200.0
2010-02-18,7251900,13174200.0


In [36]:
# 2015년 8월 1일부터 2018년 7월 31일까지의 데이터만 사용
data = {}
for i in raw_data.keys():
    data[i] = raw_data[i].loc['2015-08-01':'2018-07-31']

In [85]:
data['삼성전자'].head()

,Open,High,Low,Close,Adj Close,Volume,sh1_pct_change,sh5_pct_change,pct_change,sh1_close_ma_3,sh5_close_ma_3,sh1_his_vol_7,sh5_his_vol_7,sh1_his_vol_21,sh5_his_vol_21,sh1_volume,sh5_volume,Y1,Y3
Date,,,,,,,,,,,,,,,,,,,
2015-08-03,23680.0,23680.0,23320.0,23500.0,241.185562,9844550,-0.024691,0.000814,-0.008439,24420.000000,24620.000000,0.330462,0.206070,0.322405,0.282936,19579450.0,12152000.0,0,0
2015-08-04,23500.0,23740.0,23400.0,23640.0,242.622421,12844350,-0.008439,0.000000,0.005957,23833.333333,24593.333333,0.326568,0.135040,0.314871,0.282633,9844550.0,13381350.0,0,0
2015-08-05,23640.0,23640.0,23160.0,23180.0,237.901352,12802500,0.005957,0.026829,-0.019459,23613.333333,24820.000000,0.335496,0.215603,0.301013,0.297478,12844350.0,13734450.0,0,0
2015-08-06,23100.0,23120.0,22300.0,22300.0,228.869736,20199050,-0.019459,-0.038005,-0.037964,23440.000000,24720.000000,0.341812,0.308978,0.304783,0.314385,12802500.0,15419600.0,0,0
2015-08-07,22400.0,22740.0,22300.0,22720.0,233.180313,13787100,-0.037964,-0.024691,0.018834,23040.000000,24420.000000,0.377340,0.330462,0.327678,0.322405,20199050.0,19579450.0,0,0


In [37]:
# 1일 조정한 5가지의 feature
li1 = ["sh1_pct_change","sh1_close_ma_3","sh1_volume","sh1_his_vol_7","sh1_his_vol_21"]
# 5일 조정한 5가지의 feature
li5 = ["sh5_pct_change","sh5_close_ma_3","sh5_volume","sh5_his_vol_7","sh5_his_vol_21"]

## Train & Test 셋 분리

In [38]:
# 1일 데이터를 조정한 훈련, 테스트 데이터셋 만들기
train1 = {}
train1_idx = {}
test1 = {}
test1_idx = {}

# 5일 데이터를 조정한 훈련, 테스트 데이터셋 만들기
train5 = {}
train5_idx = {}
test5 = {}
test5_idx = {}

# 수익률, 3일 주가 이동평균, 거래량, 7일 변동성, 21일 변동성
# 위 5개 feature와 평균vs평균의 Y값으로 예측할 train, test set 생성
for code, df in data.items():
    train1[code] = df.iloc[:int(len(df)*0.8)][li1]
    train1_idx[code] = df.iloc[:int(len(df)*0.8)]["Y3"]    
    test1[code] = df.iloc[int(len(df)*0.8):][li1]
    test1_idx[code] = df.iloc[int(len(df)*0.8):]["Y3"]
    
    train5[code] = df.iloc[:int(len(df)*0.8)][li5]
    train5_idx[code] = df.iloc[:int(len(df)*0.8)]["Y3"]    
    test5[code] = df.iloc[int(len(df)*0.8):][li5]
    test5_idx[code] = df.iloc[int(len(df)*0.8):]["Y3"]

In [44]:
# 위와 동일한 코드, Scale 조정을 하기 위한 데이터셋 추가
train1_scale = {}
train1_idx_scale = {}
test1_scale = {}
test1_idx_scale = {}

train5_scale = {}
train5_idx_scale = {}
test5_scale = {}
test5_idx_scale = {}

for code, df in data.items():
    train1_scale[code] = df.iloc[:int(len(df)*0.8)][li1]
    train1_idx_scale[code] = df.iloc[:int(len(df)*0.8)]["Y3"]    
    test1_scale[code] = df.iloc[int(len(df)*0.8):][li1]
    test1_idx_scale[code] = df.iloc[int(len(df)*0.8):]["Y3"]
    
    train5_scale[code] = df.iloc[:int(len(df)*0.8)][li5]
    train5_idx_scale[code] = df.iloc[:int(len(df)*0.8)]["Y3"]    
    test5_scale[code] = df.iloc[int(len(df)*0.8):][li5]
    test5_idx_scale[code] = df.iloc[int(len(df)*0.8):]["Y3"]

In [45]:
train1['삼성전자'].head()

,sh1_pct_change,sh1_close_ma_3,sh1_volume,sh1_his_vol_7,sh1_his_vol_21
Date,,,,,
2015-08-03,-0.024691,24420.000000,19579450.0,0.330462,0.322405
2015-08-04,-0.008439,23833.333333,9844550.0,0.326568,0.314871
2015-08-05,0.005957,23613.333333,12844350.0,0.335496,0.301013
2015-08-06,-0.019459,23440.000000,12802500.0,0.341812,0.304783
2015-08-07,-0.037964,23040.000000,20199050.0,0.377340,0.327678


## 4개의 실험군 중 2개의 실험군 Standard Normalization 실시

In [47]:
# scikit-learn의 preprocessing 모듈 안에서 StandardScaler 객체 호출
from sklearn.preprocessing import StandardScaler

# 표준 정규화 Scale 실시
for code in data.keys():
    # Create StandardScaler object
    std_scaler1 = StandardScaler()
    std_scaler5 = StandardScaler()
    # fit
    std_scaler1.fit(train1_scale[code][li1].values)
    std_scaler5.fit(train5_scale[code][li5].values)
    # transform
    train1_scale[code][li1] = std_scaler1.transform(train1_scale[code][li1].values)
    test1_scale[code][li1] = std_scaler1.transform(test1_scale[code][li1].values)
    train5_scale[code][li5] = std_scaler5.transform(train5_scale[code][li5].values)
    test5_scale[code][li5] = std_scaler5.transform(test5_scale[code][li5].values)

In [48]:
# fit_transform을 적용하여 test set의 mean과 std로 정규화를 적용했을 때
std_scaler = StandardScaler()
test1['삼성전자'][li1] = std_scaler.fit_transform(test1['삼성전자'][li1].values)
test1['삼성전자'].head()

,sh1_pct_change,sh1_close_ma_3,sh1_volume,sh1_his_vol_7,sh1_his_vol_21
Date,,,,,
2017-12-22,-2.543971,0.780330,0.218415,0.416899,0.660755
2017-12-26,0.654199,0.533468,-0.323032,0.563198,0.644052
2017-12-27,-1.625364,-0.019503,0.269266,0.946825,0.936332
2017-12-28,1.348883,0.016704,-0.378840,1.655515,1.349283
2018-01-02,1.806610,0.224068,-0.593986,2.390017,0.850949


In [49]:
# fit 실시 후 transform을 적용하여 train set의 mean과 std로 정규화를 적용했을 때
test1_scale['삼성전자'].head()

,sh1_pct_change,sh1_close_ma_3,sh1_volume,sh1_his_vol_7,sh1_his_vol_21
Date,,,,,
2017-12-22,-2.993225,1.507612,0.678656,0.748069,1.080292
2017-12-26,0.618040,1.459368,-0.139840,0.855585,1.070141
2017-12-27,-1.955966,1.351300,0.755527,1.137513,1.247774
2017-12-28,1.402453,1.358376,-0.224203,1.658330,1.498744
2018-01-02,1.919304,1.398902,-0.549435,2.198117,1.195882


## 모델 호출

In [50]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn import metrics

# Sklearn과 XGBOOST에서 모델 호출

xgb_clf = XGBClassifier(random_state=42)
log_clf = LogisticRegression()
svm_clf = SVC()
rf_clf = RandomForestClassifier(random_state=42)
gnb_clf = GaussianNB()
knn_clf = KNeighborsClassifier()
dt_clf = DecisionTreeClassifier()

In [51]:
models=[xgb_clf,rf_clf,log_clf,svm_clf,dt_clf,gnb_clf,knn_clf]
model_name=['xgb','rf','log','svm','dt','gnb','knn']

In [75]:
from sklearn.utils.validation import NotFittedError
warnings.filterwarnings('ignore')

## 예측 함수 제작

In [63]:
# Python 오류 문구 제거 코드
import warnings
warnings.simplefilter('ignore', DeprecationWarning)


def modelfit_with_score(train, train_idx, test, test_idx):
    result = {} #Test 결과를 저장할 Dictionary 자료구조 생성
    conf_mx = {} #Confusion Matrix를 저장한 Dictionary 자료구조 생성
    for i in data.keys():
        # Train, Test Set 이름 일반화
        X_train = train[i].values
        y_train = train_idx[i].values
        X_test = test[i].values
        y_test = test_idx[i].values
        score = {} #주식별로 모델별 Test Score를 저장할 dict 생성
        confusion = {} #주식별로 모델별 Confusion Matrix를 저장할 dict 생성
        for clf, name in zip(models, model_name):
            model_score = [] #각 모델별 score가 저장될 list
            confusion_mat = [] #각 모델별 confusion matrix가 저장될 list
            clf.fit(X_train, y_train) #fitting
            y_pred = clf.predict(X_test) #predict
            try:
                y_prob = clf.predict_proba(X_test) #클래스 확률 계산
            # SVM의 경우, predict_proba 메서드를 실시하기 위해
            # 모델의 Probability를 True로 바꿔줘야한다.
            # 이를 위한 코드
            except NotFittedError:
                clf.probability = True
                y_prob = clf.predict_proba(X_test)
            # 모델 별 score 및 confusion matrix를 list에 저장
            model_score.append(metrics.accuracy_score(y_test, y_pred))
            model_score.append(metrics.precision_score(y_test, y_pred))
            model_score.append(metrics.recall_score(y_test, y_pred))
            model_score.append(metrics.f1_score(y_test, y_pred))
            model_score.append(metrics.roc_auc_score(y_test, y_prob[:,1]))
            confusion_mat.append(metrics.confusion_matrix(y_test, y_pred))
            # 모델별 score및 confusion matrix를 모델별로 저장
            score[name] = model_score
            confusion[name] = confusion_mat
        # 주식별 score와 confusion matrix를 저장
        result[i] = score
        conf_mx[i] = confusion
        
    # 결과값을 return으로 받음
    return result, conf_mx

def print_result(result):
    for i in data.keys():
        print(i)
        print(pd.DataFrame(result[i],index=['accuracy', 'precision', 'recall', 'f1', 'auc']).apply(lambda x : round(x,4)*100),"\n")

# 실험 개시
- predictor(feature) : 수익률, 거래량, 3일 주가 이동평균, 7일 변동성, 21일 변동성
- target : 주가의 7일 평균 vs 전일부터 7일의 주가 평균

## 첫번째 실험군 : 1일후 예측, 표준 정규화 X

In [70]:
# 1일 후를 예측, 표준 정규화 X
result1, conf_mx1 = modelfit_with_score(train1, train1_idx, test1, test1_idx)
print_result(result1)

삼성전자
              dt    gnb    knn    log     rf     svm    xgb
accuracy   59.18  49.66  49.66  59.18  49.66   50.34  54.42
precision  57.95   0.00   0.00  58.97  50.00   50.34  60.00
recall     68.92   0.00   0.00  62.16  74.32  100.00  28.38
f1         62.96   0.00   0.00  60.53  59.78   66.97  38.53
auc        59.12  62.46  50.00  62.37  51.77   50.00  53.47 

현대차
              dt    gnb    knn    log     rf     svm    xgb
accuracy   57.14  53.74  54.42  39.46  52.38   46.94  48.98
precision  53.75  50.54  51.28  36.49  49.28   46.94  46.34
recall     62.32  68.12  57.97  39.13  49.28  100.00  55.07
f1         57.72  58.02  54.42  37.76  49.28   63.89  50.33
auc        57.44  54.25  47.02  36.05  54.66   50.00  53.60 

PSOCO
              dt    gnb    knn    log     rf    svm    xgb
accuracy   62.59  58.50  63.27  57.14  61.22  49.66  65.99
precision  60.92  59.70  62.20  56.32  61.64   0.00  65.00
recall     71.62  54.05  68.92  66.22  60.81   0.00  70.27
f1         65.84  56.74  

## 두번째 실험군 : 5일 후를 예측, 표준 정규화 X

In [72]:
# 5일 후를 예측, 표준 정규화 X
result5, conf_mx5 = modelfit_with_score(train5, train5_idx, test5, test5_idx)
print_result(result5)

삼성전자
              dt    gnb    knn     log     rf     svm    xgb
accuracy   64.63  48.98  46.94   52.38  63.95   50.34  62.59
precision  62.50  49.18  47.96   51.39  64.00   50.34  71.11
recall     74.32  40.54  63.51  100.00  64.86  100.00  43.24
f1         67.90  44.44  54.65   67.89  64.43   66.97  53.78
auc        64.56  59.20  44.08   50.98  67.75   50.00  65.35 

현대차
              dt    gnb    knn    log     rf     svm    xgb
accuracy   55.78  57.82  54.42  46.94  57.82   46.94  64.63
precision  52.27  57.78  51.25  41.82  53.93   46.94  58.25
recall     66.67  37.68  59.42  33.33  69.57  100.00  86.96
f1         58.60  45.61  55.03  37.10  60.76   63.89  69.77
auc        56.41  58.58  54.94  47.27  62.86   50.00  67.22 

PSOCO
              dt    gnb    knn    log     rf    svm    xgb
accuracy   48.98  53.74  53.74  53.74  48.30  49.66  53.06
precision  49.44  54.69  54.17  53.85  48.44   0.00  53.33
recall     59.46  47.30  52.70  56.76  41.89   0.00  54.05
f1         53.99  5

## 세번째 실험군 : 1일 후를 예측, 표준 정규화 O

In [73]:
# 1일 후를 예측, 표준 정규화 O
result1s, conf_mx1s = modelfit_with_score(train1_scale, train1_idx_scale, test1_scale, test1_idx_scale)
print_result(result1s)

삼성전자
              dt    gnb    knn    log     rf    svm    xgb
accuracy   51.02  51.70  65.99  51.02  53.74  63.27  54.42
precision  51.28  51.81  63.04  51.32  55.77  59.26  54.55
recall     54.05  58.11  78.38  52.70  39.19  86.49  56.76
f1         52.63  54.78  69.88  52.00  46.03  70.33  55.63
auc        51.00  54.87  68.51  58.05  58.17  62.99  61.64 

현대차
              dt    gnb    knn    log     rf    svm    xgb
accuracy   53.74  58.50  60.54  63.95  52.38  58.50  48.98
precision  50.62  55.88  57.33  61.11  49.28  55.41  46.34
recall     59.42  55.07  62.32  63.77  49.28  59.42  55.07
f1         54.67  55.47  59.72  62.41  49.28  57.34  50.33
auc        54.07  58.23  62.40  66.65  54.66  63.34  53.60 

PSOCO
              dt    gnb    knn    log     rf    svm    xgb
accuracy   57.82  60.54  61.22  60.54  61.22  67.35  65.99
precision  57.14  60.53  60.00  59.30  61.64  66.25  65.00
recall     64.86  62.16  68.92  68.92  60.81  71.62  70.27
f1         60.76  61.33  64.15  63.75

## 네번째 실험군 : 5일 후를 예측, 표준 정규화 O

In [74]:
# 5일 후를 예측, 표준 정규화 O
result5s, conf_mx5s = modelfit_with_score(train5_scale, train5_idx_scale, test5_scale, test5_idx_scale)
print_result(result5s)

삼성전자
              dt    gnb    knn    log     rf    svm    xgb
accuracy   64.63  60.54  57.82  61.22  63.95  61.22  62.59
precision  62.79  62.90  60.00  63.93  64.00  66.67  71.11
recall     72.97  52.70  48.65  52.70  64.86  45.95  43.24
f1         67.50  57.35  53.73  57.78  64.43  54.40  53.78
auc        64.57  65.57  62.02  66.14  67.75  64.72  65.35 

현대차
              dt    gnb    knn    log     rf    svm    xgb
accuracy   52.38  64.63  58.50  64.63  57.82  61.22  65.31
precision  49.44  66.67  54.44  62.69  53.93  56.52  58.82
recall     63.77  49.28  71.01  60.87  69.57  75.36  86.96
f1         55.70  56.67  61.64  61.76  60.76  64.60  70.18
auc        53.04  69.86  62.96  68.75  62.77  68.52  67.28 

PSOCO
              dt    gnb    knn    log     rf    svm    xgb
accuracy   48.30  50.34  53.06  59.86  48.30  57.82  53.06
precision  48.86  50.70  53.33  61.19  48.44  60.00  53.33
recall     58.11  48.65  54.05  55.41  41.89  48.65  54.05
f1         53.09  49.66  53.69  58.16

# Summary
- 데이터를 재희형과 Cross Check하여 정확한 구성 실시
- Standard Scale하는 부분도 정확하게 Train의 평균과 분산으로 실시
- 4개의 실험군을 나눠 실험 개시
- modelfit_with_score 함수를 만들어 예측 실시
  - 이 함수는 fit-predict의 역할을 수행
  - 함수가 길어지게 된 이유는 보조적인 score를 저장하는 과정에서 길어지게 됨
  - 이에 대한 설명을 위에 함수에 요약 설명

# Conclusion
- 코드를 체크했음에도 불구하고 오히려 더 낮은 예측 정확도를 보임
- SVM의 경우 일전에 본 Precision이 0되는 경우는 Scale 조정을 하지 않아서 예측을 못했던 것으로 판명
  - 실제로 SVM이 Best Estimator가 되는 경우도 발견됨
  - 이를 통해 SVM으로도 Performance를 향상시킬 수 있을 것으로 조명됨

# 향후..?
- 우리가 사용하는 Y가 실제로 돈을 벌 수 있는, 유용한 Information인지 확실치 않음
- Data에 문제가 존재하지 않았고 이는 feature가 제대로 갖춰지지 않음을 의미
- Grid Search로 돌리지는 않았으나 70~80%로 끌어올리는 것에는 무리가 있을 것으로 판단
- 새로운 Methodology(Feature 및 target 선정)를 도입하는 것도 방법이라 사려됨